In [1]:
pip install dash dash-bootstrap-components pandas plotly

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import Libraries
import os
import pandas as pd
import requests
import matplotlib.pyplot as plt
import plotly.express as px
from io import StringIO
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import pickle
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objs as go

In [3]:
# Load the processed data (Replace with your actual data path)
data_path = "data/processed/subjective_wellbeing_cleaned.csv"
processed_data = pd.read_csv(data_path)

# Dash app initialization
app = dash.Dash(__name__)

# Dropdown options for Subtopics, Age Group, Gender, Suburb
subtopic_options = [{"label": sub, "value": sub} for sub in processed_data['Subtopics'].unique()]
age_group_options = [{"label": sub, "value": sub} for sub in processed_data[processed_data['Category'] == 'Age Group']['Subcategory'].unique()]
gender_options = [{"label": sub, "value": sub} for sub in processed_data[processed_data['Category'] == 'Gender']['Subcategory'].unique()]
suburb_options = [{"label": sub, "value": sub} for sub in processed_data[processed_data['Category'] == 'Suburb']['Subcategory'].unique()]

# App layout
app.layout = html.Div([
    html.H1("Subjective Wellbeing Dashboard", style={'textAlign': 'center'}),
    html.Div([
        html.Div([
            html.Label("Select Subtopic:"),
            dcc.Dropdown(
                id='subtopic-dropdown',
                options=subtopic_options,
                value=processed_data['Subtopics'].unique()[0]  # Default value
            )
        ], style={'width': '25%', 'display': 'inline-block'}),

        html.Div([
            html.Label("Select Age Group:"),
            dcc.Dropdown(
                id='age-group-dropdown',
                options=age_group_options,
                value=processed_data[processed_data['Category'] == 'Age Group']['Subcategory'].unique()[0]
            )
        ], style={'width': '25%', 'display': 'inline-block'}),

        html.Div([
            html.Label("Select Gender:"),
            dcc.Dropdown(
                id='gender-dropdown',
                options=gender_options,
                value=processed_data[processed_data['Category'] == 'Gender']['Subcategory'].unique()[0]
            )
        ], style={'width': '25%', 'display': 'inline-block'}),

        html.Div([
            html.Label("Select Suburb:"),
            dcc.Dropdown(
                id='suburb-dropdown',
                options=suburb_options,
                value=processed_data[processed_data['Category'] == 'Suburb']['Subcategory'].unique()[0]
            )
        ], style={'width': '25%', 'display': 'inline-block'}),
    ], style={'marginBottom': '20px'}),

    dcc.Graph(id='trend-graph')  # Line graph
])


# Callback for updating the graph based on dropdown values
@app.callback(
    Output('trend-graph', 'figure'),
    [
        Input('subtopic-dropdown', 'value'),
        Input('age-group-dropdown', 'value'),
        Input('gender-dropdown', 'value'),
        Input('suburb-dropdown', 'value')
    ]
)
def update_graph(selected_subtopic, selected_age_group, selected_gender, selected_suburb):
    # Filter data based on dropdown selections
    filtered_data = processed_data[
        (processed_data['Subtopics'] == selected_subtopic) &
        (
            (processed_data['Category'] == 'Age Group') & (processed_data['Subcategory'] == selected_age_group) |
            (processed_data['Category'] == 'Gender') & (processed_data['Subcategory'] == selected_gender) |
            (processed_data['Category'] == 'Suburb') & (processed_data['Subcategory'] == selected_suburb)
        )
    ]

    if filtered_data.empty:
        return go.Figure()

    # Prepare historical data for the graph
    historical_data = filtered_data[filtered_data['year'] <= 2023]
    years = historical_data['year'].values
    percentages = historical_data['Percentage'].values

    # Predictions for 2024 and 2025 using linear regression (example)
    if len(years) > 1:  # Ensure there's enough data to predict
        slope, intercept = np.polyfit(years, percentages, 1)
        pred_2024 = slope * 2024 + intercept
        pred_2025 = slope * 2025 + intercept
    else:
        pred_2024, pred_2025 = None, None

    # Create the plotly figure
    figure = go.Figure()

    # Add historical data
    figure.add_trace(go.Scatter(
        x=historical_data['year'],
        y=historical_data['Percentage'],
        mode='lines+markers',
        name='Historical Data',
        line=dict(color='blue'),
    ))

    # Add predictions
    if pred_2024 and pred_2025:
        figure.add_trace(go.Scatter(
            x=[2024, 2025],
            y=[pred_2024, pred_2025],
            mode='markers+text',
            name='Predictions',
            text=[f"{pred_2024:.2f}%", f"{pred_2025:.2f}%"],
            textposition="top center",
            marker=dict(color='red', size=10),
        ))

    # Update layout
    figure.update_layout(
        title=f"Trend for {selected_subtopic} ({selected_age_group}, {selected_gender}, {selected_suburb})",
        xaxis_title="Year",
        yaxis_title="Percentage",
        legend=dict(x=0, y=-0.2, orientation="h"),
        margin=dict(l=40, r=40, t=40, b=40)
    )

    return figure


if __name__ == '__main__':
    app.run_server(debug=True, port=2022)  


